In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
from sklearn.metrics import r2_score , mean_squared_error

In [ ]:
train_data=pd.read_csv('../input/bigmart-sales-data/Train.csv')
test_data=pd.read_csv('../input/bigmart-sales-data/Test.csv')

In [ ]:
train_data.head()

Data Dictionary:

Item_Identifier : Unique code for each item
Item_Weight : Weight of an Item
Fat_Content : Low Fat or Regular Fat
Item_Visibility : Shows percentage distribution
Item_Type : Type of an Item based on usage
MRP : Maximum Retail Price
Outlet_Identifier : Outlet number
Outlet_Establishment_Year : When outlet was started
Outlet_Size : It should be based on number of products store keeps
Location:Location of Outlet that surely impact sales (demand of a product)
Outlet Type : Type of Outlet, There has to be some relation between outlet size and type of outlet
Item Outlet Sales : Sale of an Item in particular outlet

In [ ]:
print("Train Data RowsxColumns",train_data.shape)
print("Test Data RowsxColumns",test_data.shape)
#shape provides rows vs columns information

Step 1 : Data Exploring, Understanding & Cleaning

In [ ]:
#Check for duplicate records
print("Number of Dupliate Records in Train Data : ",train_data.duplicated().sum())
print("Number of Dupliate Records in Test Data : ",test_data.duplicated().sum())

In [ ]:
#let's check if there are missing values
train_data.isnull().sum()
#Item_Weight and Outlet_Size have many missing values

In [ ]:
train_data.isnull().sum()/train_data.shape[0]
#in terms of % it's 17.16 and 28.27 resepectively

In [ ]:
#let's check test data as well
test_data.isnull().sum()/test_data.shape[0]
#Both Test and Train datasets have similar % of missing values

Treatment of missing values : 
In the next step let's apply the best treatment for missing values
Missing Values Treatments :
There can be different reasons for missing values :
--> MAR = Missing At Random - It is not that data is missing but missing of observations
--> MCaR = Missing Completely at Random = Missing data has nothing to do with hypothetical value or with other variable values
--> MNAR = Missing Not At Random = Missing because of some other variable's value or with any hypothetical value

In [ ]:
#Important Point - Item_Weight can be unique per Item
#let's check if that is the case then we can assign Item Weight if available
train_df1=pd.DataFrame(train_data,columns=['Item_Identifier','Item_Weight'])
train_df1=train_df1.dropna()

In [ ]:
#check=train_df1.groupby('Item_Identifier')['Item_Weight'].nunique()
check.max()

In [ ]:
train_df1=train_df1.drop_duplicates()

In [ ]:
use_data=pd.merge(train_data,train_df1,on="Item_Identifier")

In [ ]:
use_data.isnull().sum()

In [ ]:
#four records missing as they do not have any records availablem for weight
#let's do the same treatment for test data
test_df1=pd.DataFrame(test_data,columns=['Item_Identifier','Item_Weight'])
test_df1=test_df1.dropna()

In [ ]:
test_df1=test_df1.drop_duplicates()
use_test=pd.merge(test_data,test_df1,on="Item_Identifier")

In [ ]:
use_test.isnull().sum()

In [ ]:
#Dropping original weight column
use_data.drop('Item_Weight_x',axis=1,inplace=True)
use_test.drop('Item_Weight_x',axis=1,inplace=True)

In [ ]:
#For Outlet Size, values missing at random and we can use mode to replace fill missing values
use_data['Outlet_Size']=use_data['Outlet_Size'].fillna(use_data['Outlet_Size'].mode()[0])
use_test['Outlet_Size']=use_test['Outlet_Size'].fillna(use_test['Outlet_Size'].mode()[0])

In [ ]:
use_data.isnull().sum()

In [ ]:
use_test.isnull().sum()

In [ ]:
#start checking columns
use_data['Item_Fat_Content'].unique()

In [ ]:
#There are only two bifurcations for fat types - Regular or Low Fat
rep={'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'}
use_data['Item_Fat_Content'].replace(rep,regex=True,inplace=True)
use_test['Item_Fat_Content'].replace(rep,regex=True,inplace=True)

In [ ]:
sn.countplot(use_data['Item_Fat_Content'])
#Low Fat counts are more than Regular Fat Items
#Let's check the same with sales

In [ ]:
#Bivariate Analysis
plt.figure(figsize=(8,5))
sn.barplot('Item_Fat_Content','Item_Outlet_Sales',data=use_data,palette='mako')
#See here, Regular Fat has slighlty more sales than Low Fat, MRP can be one of the reasons

In [ ]:
#Item Types, let's see the items under different categories
plt.figure(figsize=(20,7))
sn.countplot(use_data['Item_Type'])

#Fruits & Vegetables , Snack Foods have highest number of counts

In [ ]:
plt.figure(figsize=(18,5))
sn.barplot('Item_Type','Item_Outlet_Sales',data=use_data,palette='mako')
#Sales pattern are not having that much deviation which is good

In [ ]:
sn.countplot(use_data['Outlet_Establishment_Year'])
#year 1998 has a dip, let's compare with sales

In [ ]:
plt.figure(figsize=(10,5))
sn.barplot('Outlet_Establishment_Year','Item_Outlet_Sales',data=use_data,palette='mako')
#1998 year has quite low sales compare to any other year, this might be considerd as outlier 

In [ ]:
sn.countplot(use_data['Outlet_Type'])
#Supermarket Type is way too large and out of this patter when compared with other types

In [ ]:
sn.countplot(use_data['Outlet_Size'])

Data Encoding - As we know that categorical variable can not work with Model, we need to encode them in a way that we can use in model for analysis or predicting dependent variable

We will do this process for Train and Test datasets to keep them aligned

In [ ]:
rep={'Small':0,'Medium':1,'High':2}
use_data['Outlet_Size'].replace(rep,regex=True,inplace=True)
use_test['Outlet_Size'].replace(rep,regex=True,inplace=True)

In [ ]:
rep1={'Tier 1':1,'Tier 2':2,'Tier 3':3}
use_data['Outlet_Location_Type'].replace(rep1,regex=True,inplace=True)
use_test['Outlet_Location_Type'].replace(rep1,regex=True,inplace=True)

In [ ]:
#let's check out Item Visibility 
#Item Visibiity seems a distribution or presence of an item in store
plt.figure(figsize=(8,5))
plt.scatter('Item_Visibility','Item_Outlet_Sales',data=use_data)

#It's weird that item having 0 Visibility has sales, it might be missing information
#For now we can replace it with mean

In [ ]:
use_data['Item_Visibility']=use_data['Item_Visibility'].replace(0,use_data['Item_Visibility'].mean())
use_test['Item_Visibility']=use_test['Item_Visibility'].replace(0,use_test['Item_Visibility'].mean())

In [ ]:
#Item MRP
plt.figure(figsize=(8,5))
plt.scatter(y='Item_Outlet_Sales',x='Item_MRP',data=use_data)
#higher the MRP, higher the sales

In [ ]:
#let's convert Outlet Establishment Year into numbers to be used
use_data['Years']=2020-use_data['Outlet_Establishment_Year']
use_test['Years']=2020-use_test['Outlet_Establishment_Year']

In [ ]:
dmy1=pd.get_dummies(use_data['Item_Fat_Content'])
use_data=pd.concat([use_data,dmy1],axis=1)
dmy2=pd.get_dummies(use_test['Item_Fat_Content'])
use_test=pd.concat([use_test,dmy2],axis=1)

In [ ]:
use_test=use_test.drop_duplicates()

In [ ]:
dmy3=pd.get_dummies(use_data['Outlet_Identifier'])
use_data=pd.concat([use_data,dmy3],axis=1)
dmy4=pd.get_dummies(use_test['Outlet_Identifier'])
use_test=pd.concat([use_test,dmy4],axis=1)

In [ ]:
dmy5=pd.get_dummies(use_data['Item_Type'])
use_data=pd.concat([use_data,dmy5],axis=1)
dmy6=pd.get_dummies(use_test['Item_Type'])
use_test=pd.concat([use_test,dmy6],axis=1)

In [ ]:
#to input in model, let's use all numeric columns including dummies we created
input_train=use_data.drop(['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Establishment_Year','Outlet_Type','Outlet_Identifier','Item_Outlet_Sales'],axis=1)
#input_test=use_test.drop(['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Establishment_Year','Outlet_Type','Outlet_Identifier'],axis=1)

In [ ]:
y=use_data['Item_Outlet_Sales']
x=input_train

Building A Model are training the same

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=2)

In [ ]:
reg.fit(x_train,y_train)

In [ ]:
print("intercept :", reg.intercept_ )
print("slope : ",reg.coef_)

In [ ]:
y_pred=reg.predict(x_test)

In [ ]:
score=r2_score(y_test,y_pred)
print(score)

#56% of the variance is explained by this model